In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import gsw
from scipy import integrate

In [2]:
salinity = xr.open_mfdataset('D:/ENSO_salinity/data/IAP_salinity/Salinity data orgin/*')
salinity = salinity.salinity
salinity = salinity[120:]
temp = xr.open_mfdataset('D:\ENSO_salinity\data\IAP_temperature\Temperature data orgin/*')
temp = temp.temp
temp = temp[120:]
#pressure = gsw.conversions.p_from_z(-salinity.depth_std,salinity.lat)

In [3]:
saline_contraction_coefficient = gsw.beta_const_t_exact(salinity,temp,pressure)
#saline_contraction_coefficient.to_netcdf('D:/ENSO_salinity/data/sea_level/saline_contraction_coefficient.nc')
thermal_expansion = gsw.alpha_wrt_t_exact(salinity,temp,pressure)
#thermal_expansion.to_netcdf('D:/ENSO_salinity/data/sea_level/thermal_expansion_coefficient.nc')

In [6]:
salinity_anomaly = xr.open_dataset('D:/ENSO_salinity/data/IAP_salinity/IAP_salinity_anomaly_1990_2020.nc')
salinity_anomaly = salinity_anomaly.salinity
saline_contraction_coefficient.coords['time'] = pd.date_range('1990-1',freq='M',periods=372)
saline_contraction_coefficient.load()
salinity_anomaly.load()
hsla = saline_contraction_coefficient*salinity_anomaly
hsla = -hsla
hsla_2000 = hsla.isel(depth_std=0).copy()
hsla_2000.values = integrate.trapz(hsla,hsla.coords['depth_std'])
hsla.to_netcdf('D:/ENSO_salinity/data/sea_level/halosteric_sea_level_anomaly_1990_2020.nc')
hsla_2000.to_netcdf('D:/ENSO_salinity/data/sea_level/halosteric_sea_level_2000_anomaly_1990_2020.nc')

In [4]:
temp_anomaly = xr.open_dataset('D:\ENSO_salinity\data\IAP_temperature\IAP_temperature_anomaly_1990_2020.nc')
temp_anomaly = temp_anomaly.temp
thermal_expansion.coords['time'] = pd.date_range('1990-1',freq='M',periods=372)
thermal_expansion.load()
temp_anomaly.load()
tsla = thermal_expansion*temp_anomaly
tsla_2000 = tsla.isel(depth_std=0).copy()
tsla_2000.values = integrate.trapz(tsla,tsla.coords['depth_std'])
tsla.to_netcdf('D:/ENSO_salinity/data/sea_level/thermosteric_sea_level_anomaly_1990_2020.nc')
tsla_2000.to_netcdf('D:/ENSO_salinity/data/sea_level/thermosteric_sea_level_2000_anomaly_1990_2020.nc')

In [2]:
halo = xr.open_dataset('D:/ENSO_salinity/data/sea_level/halosteric_sea_level_anomaly_1990_2020.nc')
therm = xr.open_dataset('D:/ENSO_salinity/data/sea_level/thermosteric_sea_level_anomaly_1990_2020.nc')
halo = halo.salinity
therm = therm.__xarray_dataarray_variable__
steric = therm + halo
steric.to_netcdf('D:/ENSO_salinity/data/sea_level/steric_sea_level_anomaly.nc')
steric_2000 = steric.isel(depth_std=0).copy()
steric_2000.values = integrate.trapz(steric,steric.coords['depth_std'])
steric_2000.to_netcdf('D:/ENSO_salinity/data/sea_level/steric_sea_level_2000_anomaly_1990_2020.nc')